Install Packages

In [2]:
# ✅ Install / upgrade required libraries (Vertex AI & BigQuery client)
%pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage google-cloud-aiplatform --quiet


Setup Variables

In [3]:
import os
from datetime import datetime

from google.cloud import bigquery
from google.cloud import aiplatform
from google.colab import auth

# 🔑 Authenticate to GCP (in Vertex AI Colab Enterprise this might be implicit, but this is safe)
auth.authenticate_user()

# 🔧 Set your GCP project, region, and BigQuery dataset/table names
PROJECT_ID = "qwiklabs-gcp-01-562fabefbdb6"
LOCATION = "us-central1"   # or the region where you're running Gemini
BQ_LOCATION = "US"         # BigQuery dataset location

DATASET_ID = "aurora_bay"          # choose a dataset name
TABLE_ID = "faqs"                  # original FAQs table
EMBEDDING_TABLE_ID = "faqs_embed"  # table that will include embeddings

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# Initialize Vertex AI
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Create BigQuery client
bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Load the CSV from GCS into BigQuery

In [4]:
# 📥 GCS path to the lab file
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Define dataset (create if it doesn't exist)
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = BQ_LOCATION

try:
    dataset_ref = bq_client.create_dataset(dataset_ref)  # will fail if exists
    print(f"Created dataset {dataset_ref.full_dataset_id}")
except Exception as e:
    print(f"Dataset may already exist: {e}")

# Define load job: CSV → BigQuery
table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

load_job = bq_client.load_table_from_uri(
    gcs_uri,
    table_ref,
    job_config=job_config,
)

print("⏳ Loading CSV into BigQuery...")
load_job.result()
print("✅ Load complete")

table = bq_client.get_table(table_ref)
print("Loaded rows:", table.num_rows)
print("Schema:", table.schema)


Created dataset qwiklabs-gcp-01-562fabefbdb6:aurora_bay
⏳ Loading CSV into BigQuery...
✅ Load complete
Loaded rows: 50
Schema: [SchemaField('string_field_0', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('string_field_1', 'STRING', 'NULLABLE', None, None, (), None)]


Create an embeddings table in BigQuery

In [5]:
schema = [
    bigquery.SchemaField("faq_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("question", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("answer", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("combined_text", "STRING", mode="REQUIRED"),
    # Store embedding as an ARRAY<FLOAT64>
    bigquery.SchemaField("embedding", "FLOAT64", mode="REPEATED"),
]

embedding_table_ref = bigquery.Table(
    f"{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}",
    schema=schema,
)

try:
    embedding_table_ref = bq_client.create_table(embedding_table_ref)
    print(f"✅ Created embedding table {embedding_table_ref.full_table_id}")
except Exception as e:
    print(f"Embedding table may already exist: {e}")


✅ Created embedding table qwiklabs-gcp-01-562fabefbdb6:aurora_bay.faqs_embed


Generate embeddings with Gemini and write them back to BigQuery


In [10]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import bigquery # Assuming bq_client is an instance of bigquery.Client

# 🔢 Choose an embedding model
EMBEDDING_MODEL_NAME = "text-embedding-004"

embedding_model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL_NAME)

# Assuming PROJECT_ID, DATASET_ID, TABLE_ID, EMBEDDING_TABLE_ID are defined
# and bq_client is initialized correctly:
# bq_client = bigquery.Client(project=PROJECT_ID)

# Optional: You can get the table schema if needed, but the error suggests direct column name issue
# source_table = bq_client.get_table(table_ref)

# Fetch all rows for simplicity (for larger datasets, do this in batches / with query)

# This first query is fine if it correctly identifies your Q&A columns
# You might want to remove this if the second query is meant to be the definitive one
QUESTION_COL = "string_field_0"
ANSWER_COL = "string_field_1"

# The first query was for demonstrating column names, but the second one is where the issue is.
# Let's focus on fixing the second query used for generating embeddings.

# Corrected query using the actual column names from your BigQuery table
query = f"""
SELECT
  CAST(ROW_NUMBER() OVER() AS STRING) AS faq_id,
  string_field_0 AS question,  -- Map string_field_0 to 'question' alias
  string_field_1 AS answer     -- Map string_field_1 to 'answer' alias
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
"""
faq_rows = list(bq_client.query(query))

print(f"Generating embeddings for {len(faq_rows)} rows...")

rows_to_insert = []

for row in faq_rows:
    # Access the aliased column names
    faq_id = row["faq_id"]
    question = row["question"] # Now 'question' will hold the value from string_field_0
    answer = row["answer"]     # Now 'answer' will hold the value from string_field_1
    combined_text = f"Q: {question}\nA: {answer}"

     # 🧠 Call the embeddings model
    embedding_response = embedding_model.get_embeddings([combined_text])
    embedding = embedding_response[0].values

    rows_to_insert.append(
        {
            "faq_id": faq_id,
            "question": question,
            "answer": answer,
            "combined_text": combined_text,
            "embedding": embedding,
        }
    )
# Insert into BigQuery
errors = bq_client.insert_rows_json(
    f"{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}",
    rows_to_insert,
)

if errors:
    print("❌ Errors while inserting embeddings:", errors)
else:
    print("✅ Successfully stored embeddings in BigQuery")


Generating embeddings for 50 rows...
✅ Successfully stored embeddings in BigQuery


Implement vector search in BigQuery

In [12]:
def get_query_embedding(query_text: str) -> list[float]:
    """Generate an embedding vector for the user query using the same model."""
    response = embedding_model.get_embeddings([query_text])
    return response[0].values

import numpy as np

def search_faqs(query_text: str, top_k: int = 5):
    """Return top_k most similar FAQs from BigQuery using vector similarity."""
    query_embedding = get_query_embedding(query_text)

    # BigQuery can't accept lists directly, so we pass as an array via UNNEST
    embedding_str = ", ".join(str(x) for x in query_embedding)

    # Note: we’ll compute dot product between query embedding and stored embedding
    # using ARRAY functions in BigQuery.
    search_query = f"""
    WITH query_embedding AS (
      SELECT ARRAY[{embedding_str}] AS embedding
    )
    SELECT
      f.faq_id,
      f.question,
      f.answer,
      (
        SELECT SUM(qe * fe)
        FROM UNNEST(f.embedding) AS fe
        WITH OFFSET pos
        JOIN UNNEST(q.embedding) AS qe
        WITH OFFSET pos2
        ON pos = pos2
      ) AS dot_product
    FROM `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_TABLE_ID}` AS f
    CROSS JOIN query_embedding AS q
    ORDER BY dot_product DESC
    LIMIT {top_k}
    """

    results = list(bq_client.query(search_query))
    return results


Call Gemini to answer using retrieved context

In [13]:
from vertexai.generative_models import GenerativeModel

RESPONSE_MODEL_NAME = "gemini-2.5-flash"
chat_model = GenerativeModel(RESPONSE_MODEL_NAME)

def build_context_from_results(results) -> str:
    """Combine retrieved FAQs into a context string for Gemini."""
    context_chunks = []
    for r in results:
        context_chunks.append(
            f"FAQ ID: {r['faq_id']}\nQuestion: {r['question']}\nAnswer: {r['answer']}"
        )
    return "\n\n---\n\n".join(context_chunks)





/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [14]:
def answer_with_rag(user_question: str) -> str:
    """Use BigQuery vector search + Gemini to answer the user question."""
    # 1️⃣ Retrieve top similar FAQs
    search_results = search_faqs(user_question, top_k=5)
    if not search_results:
        return "I couldn't find any relevant information in the Aurora Bay FAQ data."

    # 2️⃣ Build context string
    context = build_context_from_results(search_results)

    # 3️⃣ Prompt Gemini with RAG-style prompt
    system_instructions = """
You are a helpful assistant answering questions about the town of Aurora Bay, Alaska.
Use ONLY the information in the 'Context' section below.
If the answer is not clearly in the context, say you don't know.
Respond in a concise and friendly way.
"""

    prompt = f"""{system_instructions}

Context:
{context}

User question:
{user_question}

Answer:
"""

    response = chat_model.generate_content(prompt)
    return response.text


In [15]:
def chatbot():
    print("Aurora Bay FAQ chatbot. Ask a question, or type 'exit' to quit.\n")
    while True:
        user_q = input("You: ")
        if user_q.lower().strip() in {"exit", "quit"}:
            print("Bye!")
            break
        answer = answer_with_rag(user_q)
        print(f"Bot: {answer}\n")

# Uncomment to run interactively in the notebook:
chatbot()


Aurora Bay FAQ chatbot. Ask a question, or type 'exit' to quit.

You: what is 1 + 1?
Bot: I don't know.

You: When was Aurora Bay founded?
Bot: Aurora Bay was founded in 1901.

You: bye
Bot: I don't know.

You: exit
Bye!
